## CycleGAN

CycleGAN is a model that aims to solve the image-to-image translation
problem. The goal of the image-to-image translation problem is to learn the
mapping between an input image and an output image using a training set of
aligned image pairs. However, obtaining paired examples isn't always feasible.
CycleGAN tries to learn this mapping without requiring paired input-output images,
using cycle-consistent adversarial networks.

- [Paper](https://arxiv.org/pdf/1703.10593.pdf)

## Setup

In [ ]:
import os
import numpy as np                
import matplotlib.pyplot as plt    

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_addons as tfa
autotune = tf.data.AUTOTUNE


This code snippet imports the following libraries: os, numpy (renamed as np), matplotlib.pyplot (renamed as plt), tensorflow (renamed as tf), and tensorflow\_addons (renamed as tfa). It also configures the autotune variable to use TensorFlow's "AUTOTUNE" option.

*   `os`: Provides functions for interacting with the operating system, particularly for file and directory-related operations.
*   `numpy`: A popular library for numerical computations in Python, commonly used for manipulating arrays and performing efficient mathematical operations.
*   `matplotlib.pyplot`: A visualization library in Python that provides a MATLAB-like interface for creating plots and visualizations.
*   `tensorflow`: An open-source machine learning library developed by Google. It is used for building and training machine learning models, especially neural networks.
*   `tensorflow.keras.layers`: A sub-module of TensorFlow's Keras API that offers pre-defined layers used for constructing neural network models.
*   `tensorflow_addons`: An additional library for TensorFlow that provides extra implementations of advanced algorithms and layers to enhance and extend TensorFlow's capabilities.
*   `autotune`: A variable set to utilize TensorFlow's "AUTOTUNE" option, which automatically selects the best performance configuration based on the execution context.


## Prepare the dataset



In [ ]:

# Define the path of our local dataset
dataset_path = "./Data"

# Function to obtain image paths
def get_image_paths(dataset_dir):
    image_paths = []
    for root, _, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                image_path = os.path.join(root, file)
                image_paths.append(image_path)

    return image_paths

# Obtain the routes of the training and test images.
train_human_paths = get_image_paths(os.path.join(dataset_path, "trainA"))
train_anime_paths = get_image_paths(os.path.join(dataset_path, "trainB"))
test_human_paths = get_image_paths(os.path.join(dataset_path, "testA"))
test_anime_paths = get_image_paths(os.path.join(dataset_path, "testB"))

# Create the datasets for training and testing
train_human_ds = tf.data.Dataset.from_generator(lambda: train_human_paths, output_types=tf.string)
train_anime_ds = tf.data.Dataset.from_generator(lambda: train_anime_paths, output_types=tf.string)
test_human_ds = tf.data.Dataset.from_generator(lambda: test_human_paths, output_types=tf.string)
test_anime_ds = tf.data.Dataset.from_generator(lambda: test_anime_paths, output_types=tf.string)



# Define the standard image size.
orig_img_size = (158, 158)
# Size of the random crops to be used during training.
input_img_size = (128, 128, 3)
# Weights initializer for the layers.
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

buffer_size = 128
batch_size = 1


def normalize_img(img):
    img = tf.cast(img, dtype=tf.float32)
    # Map values in the range [-1, 1]
    return (img / 127.5) - 1.0


def preprocess_train_image(path):
    img = tf.io.read_file(path)                               
    img = tf.image.decode_jpeg(img, channels=3)               

    # Random flip
    img = tf.image.random_flip_left_right(img)
    # Resize to the original size first
    img = tf.image.resize(img, [*orig_img_size])
    # Random crop to 256X256
    img = tf.image.random_crop(img, size=[*input_img_size])
    # Normalize the pixel values in the range [-1, 1]
    img = normalize_img(img)
    return img


def preprocess_test_image(path):   
    img = tf.io.read_file(path)                               
    img = tf.image.decode_jpeg(img, channels=3)               

    # Only resizing and normalization for the test images.
    img = tf.image.resize(img, [input_img_size[0], input_img_size[1]])
    img = normalize_img(img)
    return img

This code fragment is responsible for loading and preparing images for use in the training and testing process of the machine learning model.  First, image paths are obtained within a specific directory and datasets are created using the obtained image paths. These datasets are used for training and testing the model. Then, image preprocessing functions are defined for training and testing. These functions read the images from the paths, apply preprocessing operations such as random flipping, resizing and normalization, and return the preprocessed images.

## Create `Dataset` objects

In [ ]:
# Apply the preprocessing operations to the training data
train_human = (
    train_human_ds.map(preprocess_train_image, num_parallel_calls=autotune)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)
train_anime = (
    train_anime_ds.map(preprocess_train_image, num_parallel_calls=autotune)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)

# Apply the preprocessing operations to the test data
test_human = (
    test_human_ds.map(preprocess_test_image, num_parallel_calls=autotune)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)
test_anime = (
    test_anime_ds.map(preprocess_test_image, num_parallel_calls=autotune)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)



Here the preprocessing operations are applied together with the "cache()" method to cache the data in memory to improve performance and the "shuffle(buffer_size)" method to randomize the data with a specified buffer size. Finally, the data is batched (batch_size) using the "batch()" method.

## Visualize some samples

In [ ]:
_, ax = plt.subplots(4, 2, figsize=(10, 15))
for i, samples in enumerate(zip(train_human.take(4), train_anime.take(4))):
    human = (((samples[0][0] * 127.5) + 127.5).numpy()).astype(np.uint8)
    anime = (((samples[1][0] * 127.5) + 127.5).numpy()).astype(np.uint8)
    ax[i, 0].imshow(human)
    ax[i, 1].imshow(anime)
plt.show()


This fragment shows some examples of the images in the dataset.

## Building blocks used in the CycleGAN generators and discriminators

In [ ]:

class ReflectionPadding2D(layers.Layer):
    """Implements Reflection Padding as a layer.

    Args:
        padding(tuple): Amount of padding for the
        spatial dimensions.

    Returns:
        A padded tensor with the same type as the input tensor.
    """

    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super().__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")


def residual_block(
    x,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="valid",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    dim = x.shape[-1]
    input_tensor = x

    x = ReflectionPadding2D()(input_tensor)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = activation(x)

    x = ReflectionPadding2D()(x)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.add([input_tensor, x])
    return x


def downsample(
    x,
    filters,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x


def upsample(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    kernel_initializer=kernel_init,
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2DTranspose(
        filters,
        kernel_size,
        strides=strides,
        padding=padding,
        kernel_initializer=kernel_initializer,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x


This code fragment defines several functions and a class to implement the CycleGAN style neural network architecture.

The ReflectionPadding2D class implements the reflection padding method in two dimensions. This layer is used to add the reflection padding to the convolution input, which allows to better preserve the image features.

The residual_block function implements a residual block of the neural network. Residual blocks are important to help the network learn complex nonlinear transformations. In this case, two 2D convolutions with a ReLU activation function and an instance normalization layer are used. The reflection padding layer is also applied to the residual block input.

The downsample and upsample functions implement down-sample and up-sample layers, respectively. Downsampling is used to reduce the image size and increase the number of image features, while upsampling is used to increase the image size and reduce the number of features.

## Build the generators

The generator consists of downsampling blocks: nine residual blocks
and upsampling blocks. The structure of the generator is the following:

```
c7s1-64 ==> Conv block with `relu` activation, filter size of 7
d128 ====|
         |-> 2 downsampling blocks
d256 ====|
R256 ====|
R256     |
R256     |
R256     |
R256     |-> 9 residual blocks
R256     |
R256     |
R256     |
R256 ====|
u128 ====|
         |-> 2 upsampling blocks
u64  ====|
c7s1-3 => Last conv block with `tanh` activation, filter size of 7.
```

In [ ]:

def get_resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=9,
    num_upsample_blocks=2,
    gamma_initializer=gamma_init,
    name=None,
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = ReflectionPadding2D(padding=(3, 3))(img_input)
    x = layers.Conv2D(filters, (7, 7), kernel_initializer=kernel_init, use_bias=False)(
        x
    )
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.Activation("relu")(x)

    # Downsampling
    for _ in range(num_downsampling_blocks):
        filters *= 2
        x = downsample(x, filters=filters, activation=layers.Activation("relu"))

    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.Activation("relu"))

    # Upsampling
    for _ in range(num_upsample_blocks):
        filters //= 2
        x = upsample(x, filters, activation=layers.Activation("relu"))

    # Final block
    x = ReflectionPadding2D(padding=(3, 3))(x)
    x = layers.Conv2D(3, (7, 7), padding="valid")(x)
    x = layers.Activation("tanh")(x)

    model = keras.models.Model(img_input, x, name=name)
    return model


The function starts by creating an image input using the Keras Input class. Then, a reflection padding is performed on the image using the ReflectionPadding2D class. A 7x7 convolutional layer with a number of filters is applied, followed by instance normalization and ReLU activation.

Then, a number of downsampling blocks are applied, each reducing the spatial resolution of the image by half. After the downsampling blocks, a number of residual blocks are applied to allow for connection hops in the network.

After the residual blocks, a number of upsampling blocks are applied, each doubling the spatial resolution of the image. Finally, a final padding reflection block is applied, followed by a 7x7 convolutional layer with 3 filters and a hyperbolic tangent activation (tanh).

The model is compiled using the image input and the output generated by the last convolutional layer, and returned as an instance of the Keras Model class.

## Build the discriminators

The discriminators implement the following architecture:
`C64->C128->C256->C512`

In [ ]:

def get_discriminator(
    filters=64, kernel_initializer=kernel_init, num_downsampling=3, name=None
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = layers.Conv2D(
        filters,
        (4, 4),
        strides=(2, 2),
        padding="same",
        kernel_initializer=kernel_initializer,
    )(img_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for num_downsample_block in range(3):
        num_filters *= 2
        if num_downsample_block < 2:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(2, 2),
            )
        else:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(1, 1),
            )

    x = layers.Conv2D(
        1, (4, 4), strides=(1, 1), padding="same", kernel_initializer=kernel_initializer
    )(x)

    model = keras.models.Model(inputs=img_input, outputs=x, name=name)
    return model


# Get the generators
gen_G = get_resnet_generator(name="generator_G")
gen_F = get_resnet_generator(name="generator_F")

# Get the discriminators
disc_X = get_discriminator(name="discriminator_X")
disc_Y = get_discriminator(name="discriminator_Y")


This code fragment defines a function to create a discriminator that takes an input image and produces a single-valued output indicating whether the image is real or false. The discriminator is constructed using a series of convolutional and down-sampling layers that reduce the spatial resolution of the image and increase the number of channels. Finally, a 1x1 convolution layer is used to produce a single-valued output.

After constructing the get_discriminator function, the code also defines two ResNet generators, gen_G and gen_F, using the get_resnet_generator function. Next, the code creates two discriminators, disc_X and disc_Y, using the get_discriminator function.

## Build the CycleGAN model



In [ ]:

class CycleGan(keras.Model):
    def __init__(
        self,
        generator_G,
        generator_F,
        discriminator_X,
        discriminator_Y,
        lambda_cycle=10.0,
        lambda_identity=0.5,
    ):
        super().__init__()
        self.gen_G = generator_G
        self.gen_F = generator_F
        self.disc_X = discriminator_X
        self.disc_Y = discriminator_Y
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity

    def call(self, inputs):
        return (
            self.disc_X(inputs),
            self.disc_Y(inputs),
            self.gen_G(inputs),
            self.gen_F(inputs),
        )

    def compile(
        self,
        gen_G_optimizer,
        gen_F_optimizer,
        disc_X_optimizer,
        disc_Y_optimizer,
        gen_loss_fn,
        disc_loss_fn,
    ):
        super().compile()
        self.gen_G_optimizer = gen_G_optimizer
        self.gen_F_optimizer = gen_F_optimizer
        self.disc_X_optimizer = disc_X_optimizer
        self.disc_Y_optimizer = disc_Y_optimizer
        self.generator_loss_fn = gen_loss_fn
        self.discriminator_loss_fn = disc_loss_fn
        self.cycle_loss_fn = keras.losses.MeanAbsoluteError()
        self.identity_loss_fn = keras.losses.MeanAbsoluteError()

    def train_step(self, batch_data):
        # x is Human and y is anime
        real_x, real_y = batch_data

        # For CycleGAN, we need to calculate different
        # kinds of losses for the generators and discriminators.
        # We will perform the following steps here:
        #
        # 1. Pass real images through the generators and get the generated images
        # 2. Pass the generated images back to the generators to check if we
        #    we can predict the original image from the generated image.
        # 3. Do an identity mapping of the real images using the generators.
        # 4. Pass the generated images in 1) to the corresponding discriminators.
        # 5. Calculate the generators total loss (adverserial + cycle + identity)
        # 6. Calculate the discriminators loss
        # 7. Update the weights of the generators
        # 8. Update the weights of the discriminators
        # 9. Return the losses in a dictionary

        with tf.GradientTape(persistent=True) as tape:
            # Human to fake anime
            fake_y = self.gen_G(real_x, training=True)
            # Anime to fake human -> y2x
            fake_x = self.gen_F(real_y, training=True)

            # Cycle (Human to fake anime to fake human): x -> y -> x
            cycled_x = self.gen_F(fake_y, training=True)
            # Cycle (Anime to fake human to fake anime) y -> x -> y
            cycled_y = self.gen_G(fake_x, training=True)

            # Identity mapping
            same_x = self.gen_F(real_x, training=True)
            same_y = self.gen_G(real_y, training=True)

            # Discriminator output
            disc_real_x = self.disc_X(real_x, training=True)
            disc_fake_x = self.disc_X(fake_x, training=True)

            disc_real_y = self.disc_Y(real_y, training=True)
            disc_fake_y = self.disc_Y(fake_y, training=True)

            # Generator adverserial loss
            gen_G_loss = self.generator_loss_fn(disc_fake_y)
            gen_F_loss = self.generator_loss_fn(disc_fake_x)

            # Generator cycle loss
            cycle_loss_G = self.cycle_loss_fn(real_y, cycled_y) * self.lambda_cycle
            cycle_loss_F = self.cycle_loss_fn(real_x, cycled_x) * self.lambda_cycle

            # Generator identity loss
            id_loss_G = (
                self.identity_loss_fn(real_y, same_y)
                * self.lambda_cycle
                * self.lambda_identity
            )
            id_loss_F = (
                self.identity_loss_fn(real_x, same_x)
                * self.lambda_cycle
                * self.lambda_identity
            )

            # Total generator loss
            total_loss_G = gen_G_loss + cycle_loss_G + id_loss_G
            total_loss_F = gen_F_loss + cycle_loss_F + id_loss_F

            # Discriminator loss
            disc_X_loss = self.discriminator_loss_fn(disc_real_x, disc_fake_x)
            disc_Y_loss = self.discriminator_loss_fn(disc_real_y, disc_fake_y)

        # Get the gradients for the generators
        grads_G = tape.gradient(total_loss_G, self.gen_G.trainable_variables)
        grads_F = tape.gradient(total_loss_F, self.gen_F.trainable_variables)

        # Get the gradients for the discriminators
        disc_X_grads = tape.gradient(disc_X_loss, self.disc_X.trainable_variables)
        disc_Y_grads = tape.gradient(disc_Y_loss, self.disc_Y.trainable_variables)

        # Update the weights of the generators
        self.gen_G_optimizer.apply_gradients(
            zip(grads_G, self.gen_G.trainable_variables)
        )
        self.gen_F_optimizer.apply_gradients(
            zip(grads_F, self.gen_F.trainable_variables)
        )

        # Update the weights of the discriminators
        self.disc_X_optimizer.apply_gradients(
            zip(disc_X_grads, self.disc_X.trainable_variables)
        )
        self.disc_Y_optimizer.apply_gradients(
            zip(disc_Y_grads, self.disc_Y.trainable_variables)
        )

        return {
            "G_loss": total_loss_G,
            "F_loss": total_loss_F,
            "D_X_loss": disc_X_loss,
            "D_Y_loss": disc_Y_loss,
        }


This code snippet defines a CycleGAN model in TensorFlow as a subclass of keras.Model. The class constructor initializes the generators (gen_G and gen_F), discriminators (disc_X and disc_Y), and other parameters such as lambda values for cycle loss and identity loss. The "call" method defines the forward pass of the model, where it passes the input through the discriminators and generators, returning the outputs. The "compile" method sets up the optimizers and loss functions for training the model. The "train_step" method defines a single training step, where it performs operations for training the CycleGAN model. This includes generating fake images, calculating losses for generators and discriminators, calculating gradients, and updating the weights of the networks. And finally, the method returns a dictionary containing the calculated losses during the training step.

## Create a callback that periodically saves generated images

In [ ]:

class GANMonitor(keras.callbacks.Callback):
    """A callback to generate and save images after each epoch"""

    def __init__(self, num_img=4):
        self.num_img = num_img

    def on_epoch_end(self, epoch, logs=None):
        _, ax = plt.subplots(4, 2, figsize=(12, 12))
        for i, img in enumerate(test_human.take(self.num_img)):
            prediction = self.model.gen_G(img)[0].numpy()
            prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
            img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

            ax[i, 0].imshow(img)
            ax[i, 1].imshow(prediction)
            ax[i, 0].set_title("Input image")
            ax[i, 1].set_title("Translated image")
            ax[i, 0].axis("off")
            ax[i, 1].axis("off")

            prediction = keras.preprocessing.image.array_to_img(prediction)
            prediction.save(
                "generated_img_{i}_{epoch}.png".format(i=i, epoch=epoch + 1)
            )
        plt.show()
        plt.close()

This code is a method that is executed at the end of each epoch during the training of the deep learning model. The method generates a translated image (prediction) from each input image (img), visualizes the images generated by the model during the training process and saves each generated image in a PNG file using the Keras save() function. This allows the user to observe how the model is improving as the training progresses.

## Train the end-to-end model

In [ ]:

# Loss function for evaluating adversarial loss
adv_loss_fn = keras.losses.MeanSquaredError()

# Define the loss function for the generators
def generator_loss_fn(fake):
    fake_loss = adv_loss_fn(tf.ones_like(fake), fake)
    return fake_loss


# Define the loss function for the discriminators
def discriminator_loss_fn(real, fake):
    real_loss = adv_loss_fn(tf.ones_like(real), real)
    fake_loss = adv_loss_fn(tf.zeros_like(fake), fake)
    return (real_loss + fake_loss) * 0.5


# Create cycle gan model
cycle_gan_model = CycleGan(
    generator_G=gen_G, generator_F=gen_F, discriminator_X=disc_X, discriminator_Y=disc_Y
)

# Compile the model
cycle_gan_model.compile(
    gen_G_optimizer=keras.optimizers.legacy.Adam(learning_rate=2e-4, beta_1=0.5),
    gen_F_optimizer=keras.optimizers.legacy.Adam(learning_rate=2e-4, beta_1=0.5),
    disc_X_optimizer=keras.optimizers.legacy.Adam(learning_rate=2e-4, beta_1=0.5),
    disc_Y_optimizer=keras.optimizers.legacy.Adam(learning_rate=2e-4, beta_1=0.5),
    gen_loss_fn=generator_loss_fn,
    disc_loss_fn=discriminator_loss_fn,
)

# Callbacks
plotter = GANMonitor()
checkpoint_filepath = "./model_checkpoints/cyclegan_checkpoints.{epoch:03d}"
checkpoint_dir = os.path.dirname(checkpoint_filepath)
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True
)

In [ ]:
# Here we will train the model for 30 epochs.
cycle_gan_model.fit(
    tf.data.Dataset.zip((train_human, train_anime)),
    epochs=30,
    callbacks=[plotter, model_checkpoint_callback],
)

This code fragment defines the adversarial loss function and the loss functions of the generators and discriminators. A cycle_gan_model object representing the CycleGAN model is created and compiled, using the previously defined generators and discriminators, and callbacks are defined to display the images generated during training and to store the model weights during training. Finally, the cycle_gan_model model is trained using the training data (train_human and train_anime) for a number of epochs, in this example 30.


## Test the performance of the model.

In [ ]:

# Load the checkpoints
latest = tf.train.latest_checkpoint(checkpoint_dir)
cycle_gan_model.load_weights(latest).expect_partial()
print("Weights loaded successfully")

_, ax = plt.subplots(4, 2, figsize=(10, 15))
for i, img in enumerate(test_human.take(4)):
    prediction = cycle_gan_model.gen_G(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

    ax[i, 0].imshow(img)
    ax[i, 1].imshow(prediction)
    ax[i, 0].set_title("Input image")
    ax[i, 1].set_title("Translated image")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")

    prediction = keras.preprocessing.image.array_to_img(prediction)
    prediction.save("predicted_img_{i}.png".format(i=i))
plt.tight_layout()
plt.show()

Finally, this code fragment loads the previously saved trained model weights using tf.train.latest_checkpoint to obtain the path to the most recent weights file, generates and displays the translated images using the CycleGAN model, and saves the translated images as PNG files.